# Description

This notebooks performs QA checks for a single vendor:
   - Number of NaN data points as % of total
   - Number of data points where `volume=0` as % of total

# Imports

In [ ]:
import logging
import os

import core.config.config_ as cconconf
import core.config.config_utils as ccocouti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import helpers.hs3 as hs3
import im_v2.crypto_chassis.data.client as iccdc
import research_amp.cc.qa as ramccqa

In [ ]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

# Configs

In [ ]:
def get_cmtask2329_config() -> cconconf.Config:
    """
    Get task2329-specific config.
    """
    config = cconconf.Config()
    param_dict = {
        "data": {
            # Parameters for client initialization.
            "im_client": {
                "universe_version": "v4",
                "resample_1min": False,
                "root_dir": os.path.join(
                    hs3.get_s3_bucket_path("ck"), "reorg", "historical.manual.pq"
                ),
                "partition_mode": "by_year_month",
                "dataset": "ohlcv",
                "contract_type": "futures",
                "data_snapshot": "20220707",
                "aws_profile": "ck",
            },
            # Parameters for data query.
            "read_data": {
                "start_ts": None,
                "end_ts": None,
                "columns": ["full_symbol", "close", "volume"],
                "filter_data_mode": "assert",
            },
        },
        "column_names": {
            "full_symbol": "full_symbol",
            "close_price": "close",
        },
        "stats": {
            "threshold": 30,
        },
    }
    config = cconfig.Config.from_dict(param_dict)
    return config


config = get_cmtask2329_config()
print(config)

# Load the data

In [ ]:
client = iccdc.CryptoChassisHistoricalPqByTileClient(
    **config["data"]["im_client"]
)

In [ ]:
universe = client.get_universe()
universe

In [ ]:
data = client.read_data(universe, **config["data"]["read_data"])
data.head(3)

# QA checks

Major metric for a QA check is `"bad data [%]"` which is the sum of `"volume=0 [%]"` and `"NaNs [%]"`.

In [ ]:
vendor_name = "CryptoChassis"
binance_timestamp_stats = ramccqa.get_timestamp_stats(data, vendor_name)
binance_timestamp_stats

In [ ]:
agg_level_full_symbol = ["full_symbol"]
binance_bad_data_stats = ramccqa.get_bad_data_stats(
    data, agg_level_full_symbol, vendor_name
)
binance_bad_data_stats

In [ ]:
agg_level_full_symbol_year_month = ["full_symbol", "year", "month"]
bad_data_stats_by_year_month = ramccqa.get_bad_data_stats(
    data, agg_level_full_symbol_year_month, vendor_name
)
bad_data_stats_by_year_month

In [ ]:
_ = ramccqa.plot_bad_data_by_year_month_stats(
    bad_data_stats_by_year_month, config["stats"]["threshold"]
)